# Derivace a modely založené na derivaci

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Příklad definice funkce s povinnými a nepovinnými parametry

In [ ]:
def mocnina(zaklad,n=2):
    vysledek = zaklad**n
    return vysledek

In [ ]:
mocnina(5)

In [ ]:
mocnina(5,n=4)

In [ ]:
mocnina()

## Numerická simulace v rovnici ochlazování

Studujeme úlohu $$\frac{\mathrm dT}{\mathrm dt} = - k (T-T_\infty), \quad T(0)=T_0.$$ Pokusíme se odhadnout po malých krůčcích chování řešení. Uvnitř každého krůčku budeme rychlost považovat za konstantní a změnu teploty určíme jako součin rychlosti a času (délky časového intervalu).

### Srovnání přesného analytického řešení s numerickou simulací po krocích konečné délky.

In [ ]:
# Nastavení parametrů
tmin = 0
tmax = 10
N = 10
k = 0.5
T0 = 100
T_inf = 20

# Pomocné proměnné
t = np.linspace(tmin,tmax,N) # časová osa pro simulaci
dt = t[1]-t[0]

# Počáteční nastavení
T = np.zeros(N)  # pole pro ukládání teplot
T[0] = T0  # počáteční teplota

# Simulace po časových krocích
for i in range(1,N):
    rozdil_teplot = T[i-1] - T_inf
    rychlost_ochlazovani = k*rozdil_teplot
    zmena_teploty = - dt*rychlost_ochlazovani
    T[i] = T[i-1] + zmena_teploty

# Uložení do tabulky
df = pd.DataFrame()
df["t"] = t
df["T"] = T
df


In [ ]:
# Analytické přesné řešení
df["analyticky"] = T_inf + (T0-T_inf)*np.exp(-k*t)   
    
# Vykreslení tabulky    
df.plot(x="t")
plt.gca().set(
    xlabel="čas",
    ylabel="teplota",
    title="Ochlazování rychlostí úměrnou teplotnímu rozdílu",
    ylim=(0,None)
);


**Úkol** Abychom mohli pohodlně řešit modely pro různé parametry, můžeme obsah buňky s cyklem `for` použít ve volání funkce.

In [ ]:
def numericke_reseni(
    tmin = 0,
    tmax = 10,
    N = 10,
    k = 0.5,
    T0 = 100,
    T_inf = 20,
):
    """
    Naivní simulace Newtonova modelu ochlazování. Derivace je nahrazena
    dopřednou diferencí.

    Na vstupu funkce jsou volitelné parametry nastavující časový interval
    pro simulaci, dělení intervalu udávající jemnost skoků, koeficient 
    úměrnosti, počáteční teplota a koncová teplota.

    Výstupem je tabulka (pandas.DataFrame) se sloupci t a T pro čas a teplotu.
    """
    t = np.linspace(tmin,tmax,N) # časová osa pro simlaci
    dt = t[1]-t[0]

    # Počáteční nastavení
    T = np.zeros(N)  # pole pro ukládání teplot
    T[0] = T0  # počáteční teplota

    # Simulace po časových krocích
    for i in range(1,N):
        rozdil_teplot = T[i-1] - T_inf
        rychlost_ochlazovani = k*rozdil_teplot
        zmena_teploty = - dt*rychlost_ochlazovani
        T[i] = T[i-1] + zmena_teploty

    # Uložení do tabulky
    df = pd.DataFrame()
    df["t"] = t
    df["T"] = T

    return df

numericke_reseni()


**Úkol** Experimentujte s počtem bodů $N$ a tím i s délkou kroku. Sledujte hladkost numerického řešení a jeho odchylku od přesného analytického řešení. 

V praxi musíme mít krok dostatečně malý, aby řešení bylo hladké a přesné, ale ne moc malý, aby nás to nestálo hodně paměti, výpočetního výkonu, času a aby nehrozilo, že se při mnoha výpočtech akumulují zaokrouhlovací chyby. Zpravidla nám toto obstarají procedury pro řešení automaticky.

### Simulace pro více počátečních podmínek

Vyjdeme z předchozí simulace, ale novou simulaci spustíme v cyklu. Jeden průběh cyklu pro každou počáteční podmínku.

In [ ]:

T0_seznam = [100, 80, 60, 40] 

df = pd.DataFrame()

for T0 in T0_seznam:
    reseni = numericke_reseni(T0=T0, N=100)
    df[T0] = reseni["T"]
df["t"] = reseni["t"]

print(df.head())    


# Vykreslení tabulky    
df.plot(x="t")
plt.gca().set(
    xlabel="čas",
    ylabel="teplota",
    title="Ochlazování rychlostí úměrnou teplotnímu rozdílu",
    ylim=(0,None)
)
plt.legend(title="Počáteční podmínka"); # !!! nadpis pro legendu

#### Úkol - simulace pro měnící se koeficient úměrnosti

Udělejte simulaci s jednou počáteční podmínkou a měnící se hodnotu koeficientu $k$. Nakopírujte si sem předchozí buňku a modifikujte tak, aby byla počáteční podmínka pořád stejná a měnil se koeficient $k$. Musíte sestavit seznam hodnot pro proměnnou $k$, změnit cyklování a změnit jméno sloupce v tabulce tak, aby zachycovalo hodnotu $k$.

### Simulace pro model se zpožděním

Uvedené simulace jsou naivní metodou řešení úlohy modelující pomocí derivací nějaký proces. Pro řešení modelů s derivacemi existují mnohem vyspělejší metody, které si představíme příště. V praxi se zpravidla spoléháme na tyto naprogramované profesionály. Dovednost umět si implementovat jednoduchý řešič může sloužit pro orientační první nástřel výpočtu, nebo při potřebě řešení nějaké speciální úlohy. Jedním takovým speciálním postupem může být snaha započítat do rovnice ochlazování zpoždění. To by mohlo odpovídat regulaci teploty, kdy se změna v nastavení neprojeví okamžitě. Všichni takovou situaci známe při nastavování teploty ve sprše.

In [ ]:
def numericke_reseni_se_zpozdenim(
    tmin = 0,
    tmax = 10,
    N = 10,
    k = 0.5,
    T0 = 100,
    T_inf = 20,
    zpozdeni = 0
):
    t = np.linspace(tmin,tmax,N) # časová osa pro simlaci
    dt = t[1]-t[0]
    zpozdeni_index = int(zpozdeni/dt)

    # Počáteční nastavení
    T = np.zeros(N)  # pole pro ukládání teplot
    T[0] = T0  # počáteční teplota

    # Simulace po časových krocích
    for i in range(1,N):
        predchozi_index = max(i-1-zpozdeni_index ,0)
        rozdil_teplot = T[predchozi_index] - T_inf
        rychlost_ochlazovani = k*rozdil_teplot
        zmena_teploty = - dt*rychlost_ochlazovani
        T[i] = T[i-1] + zmena_teploty

    # Uložení do tabulky
    df = pd.DataFrame()
    df["t"] = t
    df["T"] = T

    return df

In [ ]:
seznam_zpozdeni = [0,0.1,0.5,1,2]
df = pd.DataFrame()
for zpozdeni in seznam_zpozdeni:
    reseni = numericke_reseni_se_zpozdenim(zpozdeni=zpozdeni,N=1000)
    df[zpozdeni] = reseni["T"]
df["t"] = reseni["t"]
df

In [ ]:
df.plot(x="t")